In [1]:
from dotenv import load_dotenv
import os

import pandas as pd
from openweather_api import *
from mongodb_functools import MongoDBManager
from postgresql_functools import PostgreSQLManager, AirPollution

In [2]:
mongo_manager = MongoDBManager()
postgre_manager = PostgreSQLManager()

2024-05-11 12:25:31,590 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-05-11 12:25:31,590 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-05-11 12:25:31,592 INFO sqlalchemy.engine.Engine select current_schema()
2024-05-11 12:25:31,592 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-05-11 12:25:31,593 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-05-11 12:25:31,593 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-05-11 12:25:31,594 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-11 12:25:31,597 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

In [3]:
load_dotenv(override=True)
API_key = os.getenv('OPEN_WEATHER_API_KEY')
locations = ['Canberra']
country_code = 'AU'
limit = 5
data = []

for city_name in locations:
    city_info = fetch_api_data(build_city_info_url(city_name, limit, API_key))
    store_to_json(city_info, 'CityInfo.json')

    latitudes, longitudes = extract_lat_lon(city_info, country_code)
    air_pollution_info = fetch_pollution_info_for_coordinates(latitudes, longitudes, API_key)
    store_to_json(air_pollution_info, 'AirPollutionInfo.json')

    for i in range(len(air_pollution_info)):
        data.append({
            'Date': build_date_timestamp(air_pollution_info[i]['list'][0]['dt'], timezone=0, mode='date'),
            'Location': city_name,
            'AirQualityIndex': air_pollution_info[i]['list'][0]['main']['aqi'],
            'COConcentration': air_pollution_info[i]['list'][0]['components']['co'],
            'NOConcentration': air_pollution_info[i]['list'][0]['components']['no'],
            'NO2Concentration': air_pollution_info[i]['list'][0]['components']['no2'],
            'O3Concentration': air_pollution_info[i]['list'][0]['components']['o3'],
            'SO2Concentration': air_pollution_info[i]['list'][0]['components']['so2'],
            'PM25Concentration': air_pollution_info[i]['list'][0]['components']['pm2_5'],
            'PM10Concentration': air_pollution_info[i]['list'][0]['components']['pm10'],
            'NH3Concentration': air_pollution_info[i]['list'][0]['components']['nh3']
            })

air_pollution_df = pd.DataFrame(data)
air_pollution_df.to_csv('WeatherAirPollutionAUS.csv', index=False)

In [4]:
document = air_pollution_info[0]
mongo_manager.insert_document('air_pollution', document, verbose=True)
mongo_manager.find_document('air_pollution', {'coord': {"lon": 149.1013, "lat": -35.2976}}, verbose=True)

Inserting document...
Document inserted with ID: 663f479bb5d6c82ede21e7f8
Finding document...
Document found: {'_id': ObjectId('663f461d128e32b2616d9ac7'), 'coord': {'lon': 149.1013, 'lat': -35.2976}, 'list': [{'main': {'aqi': 1}, 'components': {'co': 208.62, 'no': 0, 'no2': 2.44, 'o3': 36.12, 'so2': 0.45, 'pm2_5': 0.5, 'pm10': 0.65, 'nh3': 0.19}, 'dt': 1715422749}]}


{'_id': ObjectId('663f461d128e32b2616d9ac7'),
 'coord': {'lon': 149.1013, 'lat': -35.2976},
 'list': [{'main': {'aqi': 1},
   'components': {'co': 208.62,
    'no': 0,
    'no2': 2.44,
    'o3': 36.12,
    'so2': 0.45,
    'pm2_5': 0.5,
    'pm10': 0.65,
    'nh3': 0.19},
   'dt': 1715422749}]}

In [5]:
air_pollution_info = mongo_manager.find_document('air_pollution', {'coord': {"lon": 149.1013, "lat": -35.2976}})

structured_data = {
    'Date': build_date_timestamp(air_pollution_info['list'][0]['dt'], timezone=0, mode='date'),
    'Location': city_name,
    'AirQualityIndex': air_pollution_info['list'][0]['main']['aqi'],
    'COConcentration': air_pollution_info['list'][0]['components']['co'],
    'NOConcentration': air_pollution_info['list'][0]['components']['no'],
    'NO2Concentration': air_pollution_info['list'][0]['components']['no2'],
    'O3Concentration': air_pollution_info['list'][0]['components']['o3'],
    'SO2Concentration': air_pollution_info['list'][0]['components']['so2'],
    'PM25Concentration': air_pollution_info['list'][0]['components']['pm2_5'],
    'PM10Concentration': air_pollution_info['list'][0]['components']['pm10'],
    'NH3Concentration': air_pollution_info['list'][0]['components']['nh3']
}
air_pollution = AirPollution(**structured_data)
postgre_manager.add_record(air_pollution)
data = postgre_manager.get_record(AirPollution, Location='Canberra')
print(data)

2024-05-11 12:25:31,907 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-11 12:25:31,909 INFO sqlalchemy.engine.Engine INSERT INTO "AirPollution" ("Date", "Location", "AirQualityIndex", "COConcentration", "NOConcentration", "NO2Concentration", "O3Concentration", "SO2Concentration", "PM25Concentration", "PM10Concentration", "NH3Concentration") VALUES (%(Date)s, %(Location)s, %(AirQualityIndex)s, %(COConcentration)s, %(NOConcentration)s, %(NO2Concentration)s, %(O3Concentration)s, %(SO2Concentration)s, %(PM25Concentration)s, %(PM10Concentration)s, %(NH3Concentration)s) RETURNING "AirPollution".id
2024-05-11 12:25:31,910 INFO sqlalchemy.engine.Engine [generated in 0.00041s] {'Date': '2024-05-11', 'Location': 'Canberra', 'AirQualityIndex': 1, 'COConcentration': 208.62, 'NOConcentration': 0, 'NO2Concentration': 2.44, 'O3Concentration': 36.12, 'SO2Concentration': 0.45, 'PM25Concentration': 0.5, 'PM10Concentration': 0.65, 'NH3Concentration': 0.19}
2024-05-11 12:25:31,914 INFO sqlalchemy.